In [1]:
import torch

import meshzoo

from torchfem import Solid

# Material model

Isotropic linear elastic stiffness tensor

In [2]:
# Material model
E = 1000.0
nu = 0.3
lbd = (E * nu) / ((1.0 + nu) * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))

# E_xx, E_yy, E_zz, E_yz, E_xz, E_xy
C = torch.tensor(
    [
        [lbd + 2.0 * mu, lbd, lbd, 0.0, 0.0, 0],
        [lbd, lbd + 2.0 * mu, lbd, 0.0, 0.0, 0],
        [lbd, lbd, lbd + 2.0 * mu, 0.0, 0.0, 0],
        [0.0, 0.0, 0.0, mu, 0.0, 0],
        [0.0, 0.0, 0.0, 0.0, mu, 0],
        [0.0, 0.0, 0.0, 0.0, 0.0, mu],
    ]
)

# 3D Cube with hexaeders

In [3]:
# Generate cube
points, cells = meshzoo.cube_hexa(
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
)
nodes = torch.tensor(points)
elements = cells.tolist()

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1

box = Solid(nodes, elements, forces, displacements, constraints, C)

u, f = box.solve()

box.plot(u=u, node_property={"Disp": u})

Widget(value="<iframe src='http://localhost:62122/index.html?ui=P_0x16ce0a950_1&reconnect=auto' style='width: …

# 3D Cube with hexaeders and inelastic (e.g. thermal) strain

In [4]:
# Remove contraints and set thermal strain instead
constraints[nodes[:, 0] == 1.0, 0] = False
strains = torch.stack(len(elements)*[torch.tensor([0.01, 0.01, 0.01, 0.0, 0.0, 0.0])])
print(strains.shape)

box = Solid(nodes, elements, forces, displacements, constraints, C, strains)

u, f = box.solve()

box.plot(u=u, node_property={"Disp": u})

torch.Size([64, 6])


Widget(value="<iframe src='http://localhost:62122/index.html?ui=P_0x12b9705b0_3&reconnect=auto' style='width: …

# 3D Cube with tetrahedrons

In [5]:
# Generate cube
points, cells = meshzoo.cube_tetra(
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
)
nodes = torch.tensor(points)
elements = cells.tolist()

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1


box = Solid(nodes, elements, forces, displacements, constraints, C)

u, f = box.solve()

box.plot(u=u, node_property={"Disp": u})

Widget(value="<iframe src='http://localhost:62122/index.html?ui=P_0x16dd48d00_5&reconnect=auto' style='width: …